# Description

This notebook calculates a one-sided binomial confidence interval for the single trial success probability given the number of tests conducted ```n_trials```, the number of successful tests ```n_successes``` out of these, and a desired ```confidence_level``` in this success probability.

When faced with test results, this tells you what you should expect the real single trial success probability (if you conducted an infinite number of tests) to be. You can tune how conservative or optimistic this estimate for the real single trial success probability should be by setting the ```confidence level```. If you set the confidence level to ```0.95```, the real single trial success probability will lie within the interval calculated at least in ```95%``` of these evaluations. That means there still is at most a ```5%``` chance that the real single trial success probability will be worse than the estimated interval suggests.

__Note__
1. All probabilities are given as numbers between 0 and 1, where 0 represents "never" and 1 represents 100% certainty.
2. The confidence intervals are conservative, i.e., the coverage of this hypothesis test is ```confidence_level``` or higher. That means if you use this notebook with ```confidence_level = 0.95``` for multiple trials, then you should expect the true single trial success rate to lie outside the reported confidence interval in 1/20 times on average in the worst-case.

# Parameters

In [1]:
n_trials = 100 #number of trials conducted
n_successes = 92 #number of successful trials
confidence_level = 0.95

round_to_digits = 4

assert isinstance(n_trials, int) and n_trials > 0
assert isinstance(n_successes, int) and n_successes >= 0
assert isinstance(confidence_level, float) and 0 <= confidence_level <= 1
assert isinstance(round_to_digits, int) and round_to_digits >= 1

# Evaluation

In [2]:
import matplotlib.pyplot as plt
import mpmath as mp
import numpy as np
import scipy.optimize as opt

mp.mp.dps = 50 #use 50 decimal digits precision in mpmath calculations

def binomial_distr(n_successes, p_success, n_trials):
    return mp.binomial(n_trials, n_successes) * mp.power(mp.mpf(p_success), n_successes) * mp.power(mp.mpf(1-p_success), (n_trials-n_successes))

binomial_distr_vectorized = np.vectorize(binomial_distr) #create version of binomial distribution that can be evaluated for whole lists of inputs

def binomial_confidence(n_successes, p_success_hypothesis, n_trials):
    return float(1-binomial_distr_vectorized(np.arange(n_successes, n_trials+1), p_success_hypothesis, n_trials).sum())
    
p_min = opt.bisect(lambda p: binomial_confidence(n_successes, p, n_trials)-confidence_level, 0, 1)
p_min = mp.nstr(mp.mpf(p_min), round_to_digits)

print('The single trial success probability p_success lies within [{}, 1] with a confidence of {}'.format(p_min, confidence_level))

The single trial success probability p_success lies within [0.8603, 1] with a confidence of 0.95
